# Advanced SQL
This notebook contains my notes for "Advanced SQL" course at [Kaggle](https://www.kaggle.com/learn/advanced-sql).

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## Lesson 1: JOINs and UNIONs
As in the first course, "Intro to SQL", the concept of the following lessons will be shown using the `Pet` dataset. We have two tables
![](https://i.imgur.com/dYVwS4T.png)
In the intoduction we used mainly `INNER JOIN` which tells SQL to keep only the paired couples. Here we will cover `LEFT JOIN` and `RIGHT JOIN`. From the tables above we inffer Veronica Dunn does not have a corresponding `Pet_ID`, she does not have a pet. And, since 5 does not appear in the `Pet_ID` column, Maisie does not have an owner.

### More Kinds of JOINs
**LEFT JOIN** returns all rows where the two tables have matching entries, along with all of the rows in the left table (whether there is a match or not).

**RIGHT JOIN**, we get the matching rows, along with all rows in the right table (whether there is a match or not).

Finally, a **FULL JOIN** returns all rows from both tables. Note that in general, any row that does not have a match in both tables will have NULL entries for the missing values. You can see this in the image below.
![](https://i.imgur.com/1Dvmg8S.png)

### UNIONs
We use **UNION** to vertically concatenate columns. For example
![](https://i.imgur.com/oa6VDig.png)
The data types of both columns must be the same, but the column names can be different (So, for instance, we cannot take the UNION of the Age column from the owners table and the Pet_Name column from the pets table).

We use **UNION ALL** to include duplicate values - you'll notice that 9 appears in both the owners table and the pets table, and shows up twice in the concatenated results. In order to keep only the unique values we use **UNION DISTINCT**.

### Example
We will work on the [Hacker News](https://www.kaggle.com/hacker-news/hacker-news) dataset. First the `comments` table:

In [ ]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "comments" table
table_ref = dataset_ref.table("comments")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Along witht he `stories` table

In [ ]:
# Construct a reference to the "stories" table
table_ref = dataset_ref.table("stories")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

The query below pulls information from the `stories` and `comments` tables to create a table showing all stories posted on January 1, 2012, along with the corresponding number of comments. We use a **LEFT JOIN** so that the results include stories that didn't receive any comments.

In [ ]:
# Query to select all stories posted on January 1, 2012, with number of comments
join_query = """
             WITH c AS
             (
             SELECT parent, COUNT(*) as num_comments
             FROM `bigquery-public-data.hacker_news.comments` 
             GROUP BY parent
             )
             SELECT s.id as story_id, s.by, s.title, c.num_comments
             FROM `bigquery-public-data.hacker_news.stories` AS s
             LEFT JOIN c
             ON s.id = c.parent
             WHERE EXTRACT(DATE FROM s.time_ts) = '2012-01-01'
             ORDER BY c.num_comments DESC
             """
# NOTE: The COUNT (*) function returns the number of rows that satisfy the WHERE 
# clause of a SELECT statement. We use it here because the WHERE statement in outside
# of the quary with the COUNT.

# Run the query, and return a pandas DataFrame
join_result = client.query(join_query).result().to_dataframe()
join_result.head()

Since the results are ordered by the `num_comments` column, stories without comments appear at the end of the DataFrame. (Remember that NaN stands for "not a number".)

In [ ]:
# None of these stories received any comments
join_result.tail()

Next, we write a query to select all usernames corresponding to users who wrote stories or comments on January 1, 2014. We use UNION DISTINCT (instead of UNION ALL) to ensure that each user appears in the table at most once.

# FILL THAT PART

## Lesson 2: Analytic Functions
In the Intro to SQL micro-course, we learned how to use aggregate functions, which perform calculations based on sets of rows. Here, we will define analytic functions, which also operate on a set of rows. However, unlike aggregate functions, analytic functions return a (potentially different) value for each row in the original table.

### Syntax
To understand how to write analytic functions, we'll work with a small table containing data from two different people who are training for a race. The id column identifies each runner, the date column holds the day of the training session, and time shows the time (in minutes) that the runner dedicated to training. Say we'd like to calculate a moving average of the training times for each runner, where we always take the average of the current and previous training sessions. We can do this with the following query:
![](https://i.imgur.com/rehp8HM.png)

All analytic functions have an **OVER** clause, which defines the sets of rows used in each calculation. The **OVER** clause has three (optional) parts:

The **PARTITION BY** clause divides the rows of the table into different groups. In the query above, we divide by `id` so that the calculations are separated by runner.
The **ORDER BY** clause defines an ordering within each partition. In the sample query, ordering by the `date` column ensures that earlier training sessions appear first.
The final clause (**ROWS BETWEEN 1 PRECEDING AND CURRENT ROW**) is known as a *window frame* clause. It identifies the set of rows used in each calculation. We can refer to this group of rows as a window. (Actually, analytic functions are sometimes referred to as analytic window functions or simply window functions!)
![](https://i.imgur.com/GjiKlA7.png)

### More Window Frame Clauses
There are many ways to write window frame clauses:
* `ROWS BETWEEN 1 PRECEDING AND CURRENT ROW` - the previous row and the current row.
* `ROWS BETWEEN 3 PRECEDING AND 1 FOLLOWING` - the 3 previous rows, the current row, and the following row.
* `ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED` FOLLOWING - all rows in the partition.

### More Window Functions

#### Analytic aggregate functions
Aggregate functions take all of the values within the window as input and return a single value.
* **MIN()** (or **MAX()**) - Returns the minimum (or maximum) of input values
* **AVG()** (or **SUM()**) - Returns the average (or sum) of input values
* **COUNT()** - Returns the number of rows in the input

#### Analytic navigation functions
Navigation functions assign a value based on the value in a (usually) different row than the current row.
* **FIRST_VALUE()** (or **LAST_VALUE()**) - Returns the first (or last) value in the input
* **LEAD()** (and **LAG()**) - Returns the value on a subsequent (or preceding) row. *Note: The syntax of LAG() is LAG(var, 1) for one lag.*


#### Analytic numbering functions
Numbering functions assign integer values to each row based on the ordering.
* **ROW_NUMBER()** - Returns the order in which rows appear in the input (starting with 1)
* **RANK()** - All rows with the same value in the ordering column receive the same rank value, where the next row receives a rank value which increments by the number of rows with the previous rank value.
* **Note: these function do not need a variabels inside of them since they refer directly to the row number. See the excercise for example using RANK().**

### Example
We'll work with the San Francisco Open Data dataset. We begin by reviewing the first several rows of the bikeshare_trips table.






In [ ]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "san_francisco" dataset
dataset_ref = client.dataset("san_francisco", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "bikeshare_trips" table
table_ref = dataset_ref.table("bikeshare_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Each row of the table corresponds to a different bike trip, and we can use an analytic function to calculate the cumulative number of trips for each date in 2015.

In [ ]:
# Query to count the (cumulative) number of trips per day
num_trips_query = """
                  WITH trips_by_day AS
                  (
                  SELECT DATE(start_date) AS trip_date,
                      COUNT(*) as num_trips
                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  WHERE EXTRACT(YEAR FROM start_date) = 2015
                  GROUP BY trip_date
                  )
                  SELECT *,
                      SUM(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
                               ) AS cumulative_trips
                      FROM trips_by_day
                  """

# Run the query, and return a pandas DataFrame
num_trips_result = client.query(num_trips_query).result().to_dataframe()
num_trips_result.head()

The first part of the query first calculates the daily number of trips. Then, in the second part it uses **SUM()** as an aggragate function. Since there is no PARTITION BY clause, the entire table is treated as a single partition.
The **ORDER BY** clause orders the rows by date, where earlier dates appear first.
By setting the window frame clause to **ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW**, we ensure that all rows up to and including the current date are used to calculate the (cumulative) sum. 
The next query tracks the stations where each bike began (in `start_station_id`) and ended (in `end_station_id`) the day on October 25, 2015.

In [ ]:
# Query to track beginning and ending stations on October 25, 2015, for each bike
start_end_query = """
                  SELECT bike_number,
                      TIME(start_date) AS trip_time,
                      FIRST_VALUE(start_station_id)
                          OVER (
                               PARTITION BY bike_number
                               ORDER BY start_date
                               ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                               ) AS first_station_id,
                      LAST_VALUE(end_station_id)
                          OVER (
                               PARTITION BY bike_number
                               ORDER BY start_date
                               ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                               ) AS last_station_id,
                      start_station_id,
                      end_station_id
                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  WHERE DATE(start_date) = '2015-10-25' 
                  """

# Run the query, and return a pandas DataFrame
start_end_result = client.query(start_end_query).result().to_dataframe()
start_end_result.head()

The query uses both**FIRST_VALUE()** and **LAST_VALUE()** as analytic functions.

* The **PARTITION BY** clause breaks the data into partitions based on the `bike_number` column. Since this column holds unique identifiers for the bikes, this ensures the calculations are performed separately for each bike.
* The **ORDER BY** clause puts the rows within each partition in chronological order.
* Since the window frame clause is **ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING**, for each row, its entire partition is used to perform the calculation. (This ensures the calculated values for rows in the same partition are identical.)

## Lesson 3: Nested and Repeated Data
### Nested Data
Consider a hypothetical dataset containing information about pets and their toys. We could organize this information in two different tables (a `pets table` and a `toys` table). The `toys` table could contain a "Pet_ID" column that could be used to match each toy to the pet that owns it.

Another option in BigQuery is to organize all of the information in a single table, similar to the `pets_and_toys` table below.
![](https://i.imgur.com/wxuogYA.png)

The table `pets_and_toys` contains all the information the two separeted tables on the left contain. We refer to the "Toy" column in the `pets_and_toys` table as a **nested column**, and say that the "Name" and "Type" fields are nested inside of it.
Nested columns have type **STRUCT** (or type **RECORD**). This is reflected in the table schema below.
![](https://i.imgur.com/epXFXdb.png)

To query a column with nested data, we need to identify each field in the context of the column that contains it:
* `Toy.Name` refers to the "Name" field in the "Toy" column, and
* `Toy.Type` refers to the "Type" field in the "Toy" column.
Then, if we want to spead the information from the nested column into two different columns, we will write the query 
![](https://i.imgur.com/eE2Gt62.png)

### Repeated Data
Now consider the case where each pet can have multiple toys. In this case, to collapse this information into a single table, we need to leverage a different datatype.
![](https://i.imgur.com/S93FJTE.png)
SQL has a specific datatype for this case. It called **REPEATED**. This is reflected in the table schema below, where the mode of the "Toys" column appears as 'REPEATED'.
![](https://i.imgur.com/KlrjpDM.png)
Each entry in a repeated field is an **ARRAY**, or an ordered list of (zero or more) values with the same datatype. For instance, the entry in the "Toys" column for Moon the Dog is **[Frisbee, Bone, Rope]**, which is an **ARRAY** with three values.
When querying repeated data, we need to put the name of the column containing the repeated data inside an **UNNEST()** function as follows
![](https://i.imgur.com/p3fXPxY.png)
The **UNNEST()** command appends the table into a longer one.
![](https://i.imgur.com/8j4XK8f.png)

### Nested and repeated data
If a pet has multiple toys with names, we could store this data in a repeated nested format. 
![](https://i.imgur.com/psKtza2.png)
In the more_pets_and_toys table above, "Name" and "Type" are both fields contained within the "Toys" STRUCT, and each entry in both "Toys.Name" and "Toys.Type" is an ARRAY.
Let's look at a sample query.
![](https://i.imgur.com/DiMCZaO.png)
The query above opens the nested-repeated data from before.
Recall that in `REPRATED` data we use UNNEST() to access the data inside and that in `NESTED` data we use `var.nestedvar` syntax. Here we have both cases, then we need to use UNNEST() first and then the "var.nested" syntax.

### Example
We'll work with the [Google Analytics](https://www.kaggle.com/bigquery/google-analytics-sample) Sample dataset. It contains information tracking the behavior of visitors to the Google Merchandise store, an e-commerce website that sells Google branded items.

We begin by printing the first few rows of the `ga_sessions_20170801` table.

In [1]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "google_analytics_sample" dataset
dataset_ref = client.dataset("google_analytics_sample", project="bigquery-public-data")

# Construct a reference to the "ga_sessions_20170801" table
table_ref = dataset_ref.table("ga_sessions_20170801")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,clientId,channelGrouping,socialEngagementType
0,None,1,1501583974,1501583974,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Car...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",2248281639583218707,None,None,Organic Search,Not Socially Engaged
1,None,1,1501616585,1501616585,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",8647436381089107732,None,None,Organic Search,Not Socially Engaged
2,None,1,1501583344,1501583344,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Souther...","[{'index': 4, 'value': 'APAC'}]","[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",2055839700856389632,None,None,Organic Search,Not Socially Engaged
3,None,1,1501573386,1501573386,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 0, 'minut...",0750846065342433129,None,None,Direct,Not Socially Engaged
4,None,8,1501651467,1501651467,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 22, 'minu...",0573427169410921198,None,None,Organic Search,Not Socially Engaged


In [2]:
table.schema

[SchemaField('visitorId', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('visitNumber', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('visitId', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('visitStartTime', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('date', 'STRING', 'NULLABLE', None, ()),
 SchemaField('totals', 'RECORD', 'NULLABLE', None, (SchemaField('visits', 'INTEGER', 'NULLABLE', None, ()), SchemaField('hits', 'INTEGER', 'NULLABLE', None, ()), SchemaField('pageviews', 'INTEGER', 'NULLABLE', None, ()), SchemaField('timeOnSite', 'INTEGER', 'NULLABLE', None, ()), SchemaField('bounces', 'INTEGER', 'NULLABLE', None, ()), SchemaField('transactions', 'INTEGER', 'NULLABLE', None, ()), SchemaField('transactionRevenue', 'INTEGER', 'NULLABLE', None, ()), SchemaField('newVisits', 'INTEGER', 'NULLABLE', None, ()), SchemaField('screenviews', 'INTEGER', 'NULLABLE', None, ()), SchemaField('uniqueScreenviews', 'INTEGER', 'NULLABLE', None, ()), SchemaField('timeOnScreen', 'INTEGER', 'NULLAB

We refer to the "browser" field (which is nested in the "device" column) and the "transactions" field (which is nested inside the "totals" column) as `device.browser` and `totals.transaction` in the query below:

In [3]:
# Query to count the number of transactions per browser
query = """
        SELECT device.browser AS device_browser,
            SUM(totals.transactions) as total_transactions
        FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
        GROUP BY device_browser
        ORDER BY total_transactions DESC
        """

# Run the query, and return a pandas DataFrame
result = client.query(query).result().to_dataframe()
result.head()


,device_browser,total_transactions
0,Chrome,41.0
1,Safari,3.0
2,Firefox,1.0
3,Internet Explorer,NaN
4,UC Browser,NaN


By storing the information in the "device" and "totals" columns as STRUCTs (as opposed to separate tables), we avoid expensive JOINs. This increases performance and keeps us from having to worry about JOIN keys (and which tables have the exact data we need).

Now we'll work with the "hits" column as an example of data that is both nested and repeated. Since:

* "hits" is a STRUCT (contains nested data) and is repeated,
* "hitNumber", "page", and "type" are all nested inside the "hits" column, and
* "pagePath" is nested inside the "page" field, 
we can query these fields with the following syntax:

In [ ]:
# Query to determine most popular landing point on the website
query = """
        SELECT hits.page.pagePath as path,
            COUNT(hits.page.pagePath) as counts
        FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`, 
            UNNEST(hits) as hits
        WHERE hits.type="PAGE" and hits.hitNumber=1
        GROUP BY path
        ORDER BY counts DESC
        """

# Run the query, and return a pandas DataFrame
result = client.query(query).result().to_dataframe()
result.head()

## Lesson 4: Efficient Queries
We will use two functions to compare the efficiency of different queries:
* `show_amount_of_data_scanned()` shows the amount of data the query uses.
* `show_time_to_run()` prints how long it takes for the query to execute.


In [4]:
from google.cloud import bigquery
from time import time

client = bigquery.Client()

def show_amount_of_data_scanned(query):
    # dry_run lets us see how much data the query uses without running it
    dry_run_config = bigquery.QueryJobConfig(dry_run=True)
    query_job = client.query(query, job_config=dry_run_config)
    print('Data processed: {} GB'.format(round(query_job.total_bytes_processed / 10**9, 3)))
    
def show_time_to_run(query):
    time_config = bigquery.QueryJobConfig(use_query_cache=False)
    start = time()
    query_result = client.query(query, job_config=time_config).result()
    end = time()
    print('Time to run: {} seconds'.format(round(end-start, 3)))

Using Kaggle's public dataset BigQuery integration.


### Strategies
1) Only select the columns you want.
It is tempting to start queries with **SELECT * FROM ....** It's convenient because you don't need to think about which columns you need. But it can be very inefficient.
This is especially important if there are text fields that you don't need, because text fields tend to be larger than other fields. For example:

In [5]:
star_query = "SELECT * FROM `bigquery-public-data.github_repos.contents`"
show_amount_of_data_scanned(star_query)

basic_query = "SELECT size, binary FROM `bigquery-public-data.github_repos.contents`"
show_amount_of_data_scanned(basic_query)

Data processed: 2477.421 GB
Data processed: 2.376 GB


In this case, we see a 1000X reduction in data being scanned to complete the query, because the raw data contained a text field that was 1000X larger than the fields we might need.

2) Read less data.
Both queries below calculate the average duration (in seconds) of one-way bike trips in the city of San Francisco.

In [6]:
more_data_query = """
                  SELECT MIN(start_station_name) AS start_station_name,
                      MIN(end_station_name) AS end_station_name,
                      AVG(duration_sec) AS avg_duration_sec
                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  WHERE start_station_id != end_station_id 
                  GROUP BY start_station_id, end_station_id
                  LIMIT 10
                  """
show_amount_of_data_scanned(more_data_query)

less_data_query = """
                  SELECT start_station_name,
                      end_station_name,
                      AVG(duration_sec) AS avg_duration_sec                  
                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  WHERE start_station_name != end_station_name
                  GROUP BY start_station_name, end_station_name
                  LIMIT 10
                  """
show_amount_of_data_scanned(less_data_query)

Data processed: 0.076 GB
Data processed: 0.06 GB


Since there is a 1:1 relationship between the station ID and the station name, we don't need to use the `start_station_id` and `end_station_id` columns in the query. By using only the columns with the station IDs, we scan less data.

3) Avoid N:N JOINs.
Most of the JOINs that you have executed in this course have been 1:1 JOINs. In this case, each row in each table has at most one match in the other table.
![](https://i.imgur.com/fp7oMLq.png)
Another type of **JOIN** is an **N:1 JOIN**. Here, each row in one table matches potentially many rows in the other table.
![](https://i.imgur.com/7PxE0Mr.png)

Finally, an **N:N JOIN** is one where a group of rows in one table can match a group of rows in the other table. Note that in general, all other things equal, this type of JOIN produces a table with many more rows than either of the two (original) tables that are being JOINed.
![](https://i.imgur.com/UsNZZoz.png)

Now we'll work with an example from a real dataset. Both examples below count the number of distinct committers and the number of files in several GitHub repositories.

In [7]:
big_join_query = """
                 SELECT repo,
                     COUNT(DISTINCT c.committer.name) as num_committers,
                     COUNT(DISTINCT f.id) AS num_files
                 FROM `bigquery-public-data.github_repos.commits` AS c,
                     UNNEST(c.repo_name) AS repo
                 INNER JOIN `bigquery-public-data.github_repos.files` AS f
                     ON f.repo_name = repo
                 WHERE f.repo_name IN ( 'tensorflow/tensorflow', 'facebook/react', 'twbs/bootstrap', 'apple/swift', 'Microsoft/vscode', 'torvalds/linux')
                 GROUP BY repo
                 ORDER BY repo
                 """
show_time_to_run(big_join_query)

small_join_query = """
                   WITH commits AS
                   (
                   SELECT COUNT(DISTINCT committer.name) AS num_committers, repo
                   FROM `bigquery-public-data.github_repos.commits`,
                       UNNEST(repo_name) as repo
                   WHERE repo IN ( 'tensorflow/tensorflow', 'facebook/react', 'twbs/bootstrap', 'apple/swift', 'Microsoft/vscode', 'torvalds/linux')
                   GROUP BY repo
                   ),
                   files AS 
                   (
                   SELECT COUNT(DISTINCT id) AS num_files, repo_name as repo
                   FROM `bigquery-public-data.github_repos.files`
                   WHERE repo_name IN ( 'tensorflow/tensorflow', 'facebook/react', 'twbs/bootstrap', 'apple/swift', 'Microsoft/vscode', 'torvalds/linux')
                   GROUP BY repo
                   )
                   SELECT commits.repo, commits.num_committers, files.num_files
                   FROM commits 
                   INNER JOIN files
                       ON commits.repo = files.repo
                   ORDER BY repo
                   """

show_time_to_run(small_join_query)

Time to run: 8.564 seconds
Time to run: 3.71 seconds


The first query has a large N:N JOIN. By rewriting the query to decrease the size of the JOIN, we see it runs much faster.